In [24]:
import pandas as pd
from os import listdir, path
import re
from collections import Counter
import collections
import string
from nltk.stem import WordNetLemmatizer 

# Files: three

In [12]:
folder=r'C:\\Users\Kentt\\stempro\\hw\\y2020feb16\\learning_data\\result'
files=[(path.join(folder, f), f) for f in listdir(folder) if re.match(r'.+.csv$', f)]
files

[('C:\\\\Users\\Kentt\\\\stempro\\\\hw\\\\y2020feb16\\\\learning_data\\\\result\\learning_data_info.csv',
  'learning_data_info.csv'),
 ('C:\\\\Users\\Kentt\\\\stempro\\\\hw\\\\y2020feb16\\\\learning_data\\\\result\\learning_data_sum.csv',
  'learning_data_sum.csv'),
 ('C:\\\\Users\\Kentt\\\\stempro\\\\hw\\\\y2020feb16\\\\learning_data\\\\result\\learning_url_sum.csv',
  'learning_url_sum.csv')]

# peek at these files

In [21]:
df_info=pd.read_csv(files[0][0], nrows=12, sep='\t')
df_datasum=pd.read_csv(files[1][0], nrows=111, sep='\t') 
#df_urlsum=pd.read_csv(files[2][0], nrows=1211, sep='\t', dtype={'entries':int, 'textlen':int}) 
df_urlsum=pd.read_csv(files[2][0], sep='\t', dtype={'entries':int, 'textlen':int}) 

specials = string.punctuation
spaces = " "*len(specials)
trantab = str.maketrans(specials, spaces)
#df_urlsum['words']=df_urlsum['url'].apply(lambda x:re.split(r'[/|.|:|\-|_|?|=|\#|&|%|,)|{|}|*|!|+]', x.strip().lower()))
df_urlsum['words']=df_urlsum['url'].apply(lambda x:x.lower().translate(trantab).split())
df_urlsum

,id_url,url,entries,textlen,id_college,words
0,0,http://admissionblog.agnesscott.org/category/d...,56,1351,100000,"[http, admissionblog, agnesscott, org, categor..."
1,1,http://admissionblog.agnesscott.org/category/j...,60,1612,100000,"[http, admissionblog, agnesscott, org, categor..."
2,2,http://agnesscott.edu/admission/high-school-st...,207,5198,100000,"[http, agnesscott, edu, admission, high, schoo..."
3,3,http://agnesscott.edu/admission/index.html,283,8918,100000,"[http, agnesscott, edu, admission, index, html]"
4,4,http://agnesscott.edu/visit,212,7908,100000,"[http, agnesscott, edu, visit]"
...,...,...,...,...,...,...
67780,296,https://www.benedict.edu/rotc,105,2664,100146,"[https, www, benedict, edu, rotc]"
67781,297,https://www.benedict.edu/scholarships,128,2977,100146,"[https, www, benedict, edu, scholarships]"
67782,298,https://www.benedict.edu/student-accounts,149,3927,100146,"[https, www, benedict, edu, student, accounts]"
67783,299,https://www.benedict.edu/tuition-and-fees,144,2638,100146,"[https, www, benedict, edu, tuition, and, fees]"


# url aggregation

In [25]:
common_words=['index', 'com', 'edu', 'www','http', 'https', 'org', 'html', 'htm', 'php', 'pdf']
lm = WordNetLemmatizer() 
all_words =[item for sublist in 
            list(df_urlsum['words']) for item in sublist 
            if (len(item)>1) 
            & (item.strip() not in common_words) 
            & (item.isdigit()==False) ]
all_words=[word.strip() for word in all_words if (word[0].isdigit()==False) & (word[-1].isdigit()==False)]
all_words = list(map(lambda item: lm.lemmatize(item), all_words))

d = Counter(all_words)  

d_value = sorted(d.items(), key=lambda x: x[1], reverse=True)  
d_value={i[0]:i[1] for i in d_value}
df_d_value= pd.DataFrame({'word':[k for k in d_value], 'fre':[k for _, k in d_value.items()]})

d_key=collections.OrderedDict(sorted(d.items()))
df_dic_d_key=pd.DataFrame({'word':[k for k in d_key], 'fre':[k for _, k in d_key.items()]})
 

# to csv for convenience

In [27]:
csv_freq_file=r'C:\\Users\Kentt\\stempro\\hw\\y2020feb16\\learning_data\\result\\df_d_value.csv'
csv_word_file=r'C:\\Users\Kentt\\stempro\\hw\\y2020feb16\\learning_data\\result\\df_d_key.csv'
df_dic_d_key.to_csv(csv_word_file, index=False)
df_d_value.to_csv(csv_freq_file, index=False)
df_d_value

,word,fre
0,admission,14091
1,event,5335
2,artinstitutes,5076
3,academic,3719
4,student,3686
...,...,...
11945,waive,1
11946,logistics,1
11947,cdei,1
11948,svclearning,1
